In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

In [3]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from cleverhans import attacks
from cleverhans import utils, utils_tf, utils_keras

config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

Using TensorFlow backend.


In [4]:
from cleverhans.model import Model
from tensorflow.contrib import slim
from tensorflow.contrib.slim.nets import inception

# Data

In [5]:
cleandata = np.load("./data/clean100data.npy").astype(np.float32)
cleanlabel = np.load("./data/clean100label.npy")
targets = np.load("./data/random_targets.npy")

In [6]:
targets_expanded = np.concatenate([np.zeros((100, 1)), targets], axis=1)

# Tensorflow inception v3

In [7]:
tf.flags.DEFINE_string(
    'checkpoint_path',
    './data/inception_v3.ckpt', 'Path to checkpoint for inception network.')

In [8]:
FLAGS = tf.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel') 

In [9]:
class InceptionModel(Model):
    """Model class for CleverHans library."""

    def __init__(self, nb_classes):
        super(InceptionModel, self).__init__(nb_classes=nb_classes,
                                             needs_dummy_fprop=True)
        self.built = False

    def __call__(self, x_input, return_logits=False):
        """Constructs model and return probabilities for given input."""
        reuse = True if self.built else None
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            x_input = x_input * 2.0 - 1.0
            _, end_points = inception.inception_v3(
                x_input, num_classes=self.nb_classes, is_training=False,
                reuse=reuse)
        self.built = True
        self.logits = end_points['Logits']
        # Strip off the extra reshape op at the output
        self.probs = end_points['Predictions'].op.inputs[0]
        if return_logits:
            return self.logits
        else:
            return self.probs

    def get_logits(self, x_input):
        return self(x_input, return_logits=True)

    def get_probs(self, x_input):
        return self(x_input)

In [10]:
def _top_1_accuracy(logits, labels):
    return tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits[:, 1:], labels, 1), tf.float32))

In [11]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)

    saver = tf.train.Saver(slim.get_model_variables())

    session_creator = tf.train.ChiefSessionCreator(
        scaffold=tf.train.Scaffold(saver=saver),
        checkpoint_filename_with_path=FLAGS.checkpoint_path)

    with tf.train.MonitoredSession(session_creator=session_creator) as sess:
        acc_val = sess.run(acc, feed_dict={x_input: cleandata, y_label: cleanlabel})
        tf.logging.info('Accuracy: %s', acc_val)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Accuracy: 1.0


# Generate AEs

In [12]:
def getabatch(dataset,labelset,numbatch,batchsize):
    databatch = dataset[numbatch*batchsize:(numbatch*batchsize+batchsize)]
    labelbatch = labelset[numbatch*batchsize:(numbatch*batchsize+batchsize)]
    return databatch,labelbatch

In [13]:
def get_random_targets(label):
    targets = label.copy()
    nb_samples = label.shape[0]
    nb_classes = label.shape[1]

    for i in range(nb_samples):
        targets[i, :] = np.roll(targets[i, :], random.randint(1, nb_classes - 1))

    return targets

In [14]:
def one_hot(raw_label):
    nb_samples = len(raw_label)
    nb_classes = 1000
    label = np.zeros((nb_samples, nb_classes))
    for i in range(nb_samples):
        label[i, raw_label[i]] = 1
        
    return label

# FGSM

In [15]:
eps = 0.01
order = np.inf

In [16]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)
    
    fgsm_params = {'y_target': targets_expanded, 'eps': eps,
                   'ord': order, 'clip_min': 0., 'clip_max': 1.}
    fgsm = attacks.FastGradientMethod(model)
    # adv_fgsm = np.copy(cleandata)
    adv_fgsm = fgsm.generate(x_input, **fgsm_params)

    saver = tf.train.Saver(slim.get_model_variables())

    session_creator = tf.train.ChiefSessionCreator(
        scaffold=tf.train.Scaffold(saver=saver),
        checkpoint_filename_with_path=FLAGS.checkpoint_path)

    with tf.train.MonitoredSession(session_creator=session_creator) as sess:
        adv_image = sess.run(adv_fgsm, feed_dict={x_input: cleandata})



Instructions for updating:
dim is deprecated, use axis instead
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [17]:
np.save("./AE/FGSM_001_AE_data.npy", adv_image)

# BIM

In [18]:
eps = 0.01
order = np.inf

In [19]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)
    
    ifgsm_params = {'y_target':targets_expanded, 'eps': eps, 'eps_iter': eps / 10,
                'ord': order, 'clip_min': 0., 'clip_max': 1.}
    ifgsm = attacks.BasicIterativeMethod(model)
    adv = ifgsm.generate(x_input, **ifgsm_params)

    saver = tf.train.Saver(slim.get_model_variables())

    session_creator = tf.train.ChiefSessionCreator(
        scaffold=tf.train.Scaffold(saver=saver),
        checkpoint_filename_with_path=FLAGS.checkpoint_path)

    with tf.train.MonitoredSession(session_creator=session_creator) as sess:
        adv_image = sess.run(adv, feed_dict={x_input: cleandata})

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [20]:
np.save("./AE/IFGSM_001_AE_data.npy", adv_image)

# LBFGS

In [ ]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)
    
    lbfgs_params = {'y_target': targets_expanded, 'batch_size':100,
                'binary_search_steps': 4, 'max_iterations': 1000,
                'clip_min': 0., 'clip_max': 1.}

    saver = tf.train.Saver(slim.get_model_variables())

    session_creator = tf.train.ChiefSessionCreator(
        scaffold=tf.train.Scaffold(saver=saver),
        checkpoint_filename_with_path=FLAGS.checkpoint_path)
    
    # with tf.train.MonitoredSession(session_creator=session_creator) as sess:
    with tf.Session() as sess:
        saver.restore(sess, FLAGS.checkpoint_path)
        lbfgs = attacks.LBFGS(model, sess)
        adv = np.copy(cleandata)
        adv = lbfgs.generate_np(adv, **lbfgs_params)
        # adv_image = sess.run(adv, feed_dict={x_input: cleandata})

INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt


[INFO 2020-05-05 21:33:39,285 cleverhans] Constructing new graph for attack LBFGS


In [33]:
np.save("./AE/LBFGS_AE_data.npy",adv)

# CW

In [ ]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)

    saver = tf.train.Saver(slim.get_model_variables())
    
    # with tf.train.MonitoredSession(session_creator=session_creator) as sess:
    with tf.Session() as sess:
        saver.restore(sess, FLAGS.checkpoint_path)
        cw = attacks.CarliniWagnerL2(model, sess)
        # carlini and wagner
        cw_params = {'y_target': targets_expanded,
                     'batch_size': 100,
                     'confidence': 10,
                     'learning_rate': 0.1,
                     'binary_search_steps': 5,
                     'max_iterations': 1000,
                     'abort_early': True,
                     'initial_const': 0.01,
                     'clip_min': 0,
                     'clip_max': 1}
        
        adv = np.copy(cleandata)
        adv = cw.generate_np(adv, **cw_params)

INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt


[INFO 2020-05-05 21:38:43,615 cleverhans] Constructing new graph for attack CarliniWagnerL2





Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


/home/stanislas/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/stanislas/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
np.save("./AE/CW_AE_data.npy",adv)

# Deepfool

In [18]:
with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
    y_label = tf.placeholder(tf.int32, shape=(None,))
    model = InceptionModel(nb_classes=1001)
    logits = model.get_logits(x_input)
    acc = _top_1_accuracy(logits, y_label)

    saver = tf.train.Saver(slim.get_model_variables())
    
    # with tf.train.MonitoredSession(session_creator=session_creator) as sess:
    with tf.Session() as sess:
        saver.restore(sess, FLAGS.checkpoint_path)
        deepfool_params = {'nb_candidate': 10, 'overshoot': 0.02, 'clip_min': 0., 'clip_max': 1.}
        deepfool = attacks.DeepFool(model, sess)
        totalbatch = 10
        batchsize = 10

        adv = np.copy(cleandata)
        
        for numbatch in range(totalbatch):
            adv_df_batch,labelbatch = getabatch(adv,cleanlabel,numbatch,batchsize)
            adv_df_batch = deepfool.generate_np(adv_df_batch, **deepfool_params)
            adv[numbatch*batchsize:(numbatch*batchsize+batchsize)] = adv_df_batch

INFO:tensorflow:Restoring parameters from ./data/inception_v3.ckpt


[INFO 2020-05-05 21:48:10,704 cleverhans] Constructing new graph for attack DeepFool
[INFO 2020-05-05 21:48:35,319 cleverhans] Attack result at iteration 5 is [531 128  99 662 252 172 818 609 772 491]
[INFO 2020-05-05 21:48:37,077 cleverhans] Attack result at iteration 7 is [531 129  99 662 252 172 818 465 772 748]
[INFO 2020-05-05 21:48:37,082 cleverhans] 10 out of 10 become adversarial examples at iteration 7
[INFO 2020-05-05 21:48:41,768 cleverhans] Attack result at iteration 5 is [298 723 251 594 378 887 705 507 921 706]
[INFO 2020-05-05 21:48:46,039 cleverhans] Attack result at iteration 10 is [298 753 251 594 378 632 911 507 921 706]
[INFO 2020-05-05 21:48:46,044 cleverhans] 10 out of 10 become adversarial examples at iteration 10
[INFO 2020-05-05 21:48:50,705 cleverhans] Attack result at iteration 5 is [ 45 955 667 582 205 218 275 140 319 639]
[INFO 2020-05-05 21:48:50,708 cleverhans] 10 out of 10 become adversarial examples at iteration 5
[INFO 2020-05-05 21:48:55,471 cleverhan

In [19]:
np.save("./AE/Deepfool_AE_data.npy", adv)